# Linear models - Regularization

In this notebook, we will focus on the concept of regularization in linear models.

In [ ]:
# temporary fix to avoid spurious warning raised in scikit-learn 1.0.0
# it will be solved in scikit-learn 1.0.1
import warnings
warnings.filterwarnings("ignore", message="X has feature names.*")
warnings.filterwarnings("ignore", message="X does not have valid feature names.*")

## Introductionary example

In this first example, we will show a known issue due to correlated features when fitting a linear model.

The data generative process to create the data is a linear relationship between the features and the target. However, out of 5 features, only 2 features will be used while 3 other features will not be linked to the target. In addition, a little bit of noise will be added. When generating the dataset, we can as well get the true model.

In [ ]:
from sklearn.datasets import make_regression

data, target, coef = make_regression(
    n_samples=2_000,
    n_features=5,
    n_informative=2,
    shuffle=False,
    coef=True,
    random_state=0,
    noise=30,
)

In [ ]:
import seaborn as sns
sns.set_context("poster")

In [ ]:
import pandas as pd

feature_names = [f"Features {i}" for i in range(data.shape[1])]
coef = pd.Series(coef, index=feature_names)
coef.plot.barh()
coef

Plotting the true coefficients, we observe that only 2 features out of the 5 features as an impact on the target.

Now, we will fit a linear model on this dataset.

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(data, target)
linear_regression.coef_

In [ ]:
feature_names = [f"Features {i}" for i in range(data.shape[1])]
coef = pd.Series(linear_regression.coef_, index=feature_names)
_ = coef.plot.barh()

We observe that we can recover almost the true coefficients. The small fluctuation are due to the noise that we added into the dataset when generating it.

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    Now, we will create some correlated feature and observe what is the impact.
    <ul>
        <li>Using <tt>X</tt>, add 4 new features that will be a repetition of features #0 and #1.</li>
        <li>Fit a <tt>LinearRegression</tt> model.</li>
        <li>Plot the value of the coefficients.</li>
    </ul>
    Are the coefficient values meaningful in regarde to the true coefficients used to generate the dataset?
</div>

In [ ]:
# %load solutions/solution_10.py

In [ ]:
# %load solutions/solution_11.py

In [ ]:
# %load solutions/solution_12.py

## Ridge regressor - L2 regularization

We saw that our linear model does not succeed to recover the true coefficients. This is due to the fact that the coefficients are not constraints and that the problem is ill-posed due to the multicollinearity. We can solve this issue by imposing some constraints on the weights of the model: this is call regularization. One possible solution is to imposed an L2 regularization on the weights. The loss to be minimized becomes:

$$
loss = (y - X \beta)^2 + \alpha \|\beta\|_2
$$

This regularization will enforce the weights to shrink towards zero. The parameter controlling the shrinking is the parameter $\alpha$.
Such a regression model is known as `Ridge` in scikit-learn. Let's fit such model and check the effect on the weights.

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1)
ridge.fit(data, target)
ridge.coef_

In [ ]:
coef = pd.Series(ridge.coef_, index=feature_names)
_ = coef.plot.barh()

Applying a small regularization solves the problem of the weight. We can even find the original relationship:

In [ ]:
ridge.coef_[:5] * 3

By looking at the loss, we see that increasing $\alpha$ will shrink more the weights.

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1e5)
ridge.fit(data, target)
ridge.coef_

In [ ]:
coef = pd.Series(ridge.coef_, index=feature_names)
_ = coef.plot.barh()

We see that the weights shrinked toward zeros but the relative weights of each feature remained the same. We can now verify that passing a very small $\alpha$ will be equivalent to using a `LinearRegression`.

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1e-14)
ridge.fit(data, target)
ridge.coef_

In [ ]:
coef = pd.Series(ridge.coef_, index=feature_names)
_ = coef.plot.barh()

## Lasso regressor - L1 regularization

Another possible type of regularization is L1. It can be formalized with:

$$
loss = (y - X \beta)^2 + \alpha \|\beta\|_1
$$

This type of regressor in called `Lasso` in scikit-learn.

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    Repeat the previous experiment varying the value of $\alpha$ and check what is the impact of $\alpha$ on the weights $\beta$.
</div>

In [ ]:
# %load solutions/solution_13.py

In [ ]:
# %load solutions/solution_14.py

In [ ]:
# %load solutions/solution_15.py

In [ ]:
# %load solutions/solution_16.py

In [ ]:
# %load solutions/solution_17.py

In [ ]:
# %load solutions/solution_18.py

## Elastic net - Combining both L2 and L1 regularization

Sometimes we can be interested by combining both L2 and L1 regularization: find a subset of features that should contribute to the target prediction but, at the same time, avoid that the non-null coefficients to be arbritary large without any constraint.

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)
elastic_net.fit(data, target)
elastic_net.coef_

In [ ]:
coef = pd.Series(elastic_net.coef_, index=feature_names)
_ = coef.plot.barh()

## What about classification?

In classification, the different choices of regularization do not give rise to different estimator. Instead, the regularization can be set as a parameter of the linear model itself. In classification, there is mainly two models: `LogisticRegression` and `LinearSVC`. They vary in the fact that they minimized different losses. However, they both have a parameter `penalty` and a parameter `C` (that is the inverse of `alpha` in regression).

In this section, we will use a `LogisticRegression` and we will check the impact of the parameter `C`. Let's first load a classification dataset. Here, we want to predict the penguin species from two features that are the culmen length and depth.

In [ ]:
data = pd.read_csv("../datasets/penguins_classification.csv")
data = data[data["Species"].isin(["Adelie", "Chinstrap"])]
data["Species"] = data["Species"].astype("category")
data.head()

In [ ]:
X, y = data[["Culmen Length (mm)", "Culmen Depth (mm)"]], data["Species"]

In [ ]:
import matplotlib.pyplot as plt

_ = data.plot.scatter(
    x="Culmen Length (mm)", y="Culmen Depth (mm)", c="Species",
    cmap=plt.cm.RdBu, s=50,
)

<div class="alert alert-success">
    <p><b>QUESTION</b>:</p>
    Looking at the documentation, what is the default regularization (or no regularization) by default in <tt>LogisticRegression</tt>?
</div>

Below, we fit a `LogisticRegression` model with the default parameters.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X, y)

In [ ]:
from helper.plotting import DecisionBoundaryDisplay

display = DecisionBoundaryDisplay.from_estimator(
    model,
    X,
    response_method="decision_function",
    cmap=plt.cm.RdBu,
    plot_method="pcolormesh",
    shading="auto",
)
_ = data.plot.scatter(
    x="Culmen Length (mm)", y="Culmen Depth (mm)", c="Species",
    cmap=plt.cm.RdBu, s=50, ax=display.ax_
)

In [ ]:
coef = pd.Series(model.coef_[0], index=X.columns)
_ = coef.plot.barh()

We can use this example as a baseline to observe the effect of the parameter `C`. In addition, we can provide that the loss function of a logistic regression is the following:

$$
loss = \frac{1 - \rho}{2} w^T w + \rho \|w\|_1 + C \log ( \exp (y_i (X \beta)) + 1)
$$

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    Repeat a fit and check the impact of the paramter <tt>C</tt> on both the coefficients and the decision boundary of the classifier.
</div>

In [ ]:
# %load solutions/solution_19.py

In [ ]:
# %load solutions/solution_20.py

In [ ]:
# %load solutions/solution_21.py

In [ ]:
# %load solutions/solution_22.py

In [ ]:
# %load solutions/solution_23.py

In [ ]:
# %load solutions/solution_24.py

Looking at the loss above, we see that the parameter `C` is applied on the data term (computing the error between the true and predicted target). In regression, this regularization weight $\alpha$ was applied on the weights instead. For this reason, `C` have the inverse impact than $\alpha$